In [1]:
import pandas as pd
import numpy as np

from nba_api.stats.endpoints import leaguegamefinder

In [2]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team ID
# celtics = [team for team in nba_teams if team['abbreviation'] == 'BOS'][0]
# celtics_id = celtics['id']

In [3]:
teams = list(map(lambda x: [x['id'],x['abbreviation']], nba_teams))

In [96]:
teams_df = pd.DataFrame(data = teams, columns=['Team ID', 'Team Acronym'])

In [5]:
games_by_team = []

for team in teams:
    temp = leaguegamefinder.LeagueGameFinder(team_id_nullable=team[0]).get_data_frames()[0]
    games_by_team.append(temp)

In [11]:
def get_wins_in_last_10_games(df):
    df = df.reindex(index=df.index[::-1]).reset_index(drop=True)
    df['wl_num'] = df['WL'].replace({'W':1, 'L':0, 'O':0, 'T':0})
    df['wins_in_last_10'] = df.groupby('SEASON_ID')['wl_num'].rolling(10, min_periods=10).sum().shift(1).values
    return df

In [12]:
for i in range(len(games_by_team)):
    games_by_team[i] = get_wins_in_last_10_games(games_by_team[i])

In [28]:
def is_home(df):
    df['is_home'] = df['MATCHUP'].apply(lambda x: 0 if '@' in x else 1)
    return df

In [29]:
for i in range(len(games_by_team)):
    games_by_team[i] = is_home(games_by_team[i])

In [30]:
games_by_team[0]

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,wl_num,wins_in_last_10,is_home
0,21983,1610612737,ATL,Atlanta Hawks,0028300005,1983-10-28,ATL @ NJN,L,240,108,...,33,14,10.0,5,17,30,NaN,0.0,NaN,0
1,21983,1610612737,ATL,Atlanta Hawks,0028300014,1983-10-29,ATL vs. DET,W,240,117,...,48,28,14.0,7,23,35,NaN,1.0,NaN,1
2,21983,1610612737,ATL,Atlanta Hawks,0028300027,1983-11-01,ATL vs. WAS,W,240,95,...,41,20,7.0,10,16,34,NaN,1.0,NaN,1
3,21983,1610612737,ATL,Atlanta Hawks,0028300041,1983-11-04,ATL vs. CHI,W,240,103,...,46,31,14.0,13,18,27,NaN,1.0,NaN,1
4,21983,1610612737,ATL,Atlanta Hawks,0028300058,1983-11-06,ATL @ MIL,L,240,84,...,37,18,4.0,5,14,24,NaN,0.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3441,22021,1610612737,ATL,Atlanta Hawks,0022100407,2021-12-13,ATL vs. HOU,L,242,126,...,40,20,9.0,6,12,26,-6.0,0.0,7.0,1
3442,22021,1610612737,ATL,Atlanta Hawks,0022100418,2021-12-15,ATL @ ORL,W,240,111,...,40,25,11.0,5,13,16,12.0,1.0,6.0,0
3443,22021,1610612737,ATL,Atlanta Hawks,0022100434,2021-12-17,ATL vs. DEN,L,241,115,...,37,22,6.0,3,15,17,-18.0,0.0,5.0,1
3444,22021,1610612737,ATL,Atlanta Hawks,0022100470,2021-12-22,ATL vs. ORL,L,241,98,...,49,21,4.0,8,14,19,-6.0,0.0,5.0,1


In [31]:
all_games = pd.concat(games_by_team)

In [35]:
all_games = all_games[all_games['is_home'].astype(bool)].reset_index(drop=True)

In [80]:
wins_in_last_10 = all_games[['SEASON_ID', 'TEAM_ID', 'GAME_ID', 'wins_in_last_10']]

In [70]:
d = gamerotation.GameRotation('0022100470').get_data_frames()